# Генерация текста с использованием N-грамм

Представим, что мы читаем книгу, и чтобы понять следующее слово, мы смотрим на несколько предыдущих слов. Этот подход называется n-граммами. В n-граммах мы рассматриваем группы из n слов, чтобы предсказывать следующее слово.

Принцип работы:

1. Мы берем текст и делим его на группы из n слов. Например, если мы используем 2-граммы (группы по 2 слова), то в предложении "Кошка сидит на диване" будут такие группы:

- "Кошка сидит"
- "сидит на"
- "на диване"

2. Мы запоминаем, какие слова часто идут вместе. Например, если в тексте часто встречаются фразы "Кошка сидит" и "сидит на", мы это запомним.

3. Когда мы хотим сгенерировать новый текст, мы начинаем с какого-то слова или фразы. Затем смотрим, какие слова чаще всего следуют за этим словом или фразой, и добавляем одно из них. Повторяем этот процесс, чтобы получить целое предложение.

Рассмотрим пример генерации текста с помощью 2-грамм (би-грамм):

Обучение:

Допустим, у нас есть текст: "Кошка сидит на диване. Собака лежит на ковре."
Мы делим текст на би-граммы:

- "Кошка сидит"
- "сидит на"
- "на диване"
- "Собака лежит"
- "лежит на"
- "на ковре"

Генерация:

Теперь мы хотим сгенерировать новое предложение. Начнем с первого слова "Кошка":

- "Кошка" -> Следующее слово в нашей модели: "сидит"
- "Кошка сидит" -> Следующее слово: "на"
- "сидит на" -> Следующее слово: "диване"

Таким образом, наше новое предложение будет: "Кошка сидит на диване."

### Подготовка среды

In [1]:
from IPython.display import clear_output

!pip install nltk
clear_output()

### Импорт библиотек

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksioprime/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk import ngrams, word_tokenize
import random
from collections import defaultdict, Counter
import re

### Вставка отрывка из текста

In [4]:
text = """
Кошка сидит на диване. Собака лежит на ковре."
"""

### Обработка и токенизация текста

In [5]:
tokens = nltk.word_tokenize(text.lower(), language='russian')
tokens = [token for token in tokens if re.match(r'\w+', token)]
print(tokens)

['кошка', 'сидит', 'на', 'диване', 'собака', 'лежит', 'на', 'ковре']


### Создание N-грамм

In [6]:
n = 2
test_grams = ngrams(tokens, n)
print(list(test_grams))

[('кошка', 'сидит'), ('сидит', 'на'), ('на', 'диване'), ('диване', 'собака'), ('собака', 'лежит'), ('лежит', 'на'), ('на', 'ковре')]


### Создание N-граммной модели

In [7]:
# Функция для генерации N-грамм
def generate_ngrams(tokens, n):
    n_grams = ngrams(tokens, n)
    ngrams_freq = defaultdict(Counter)
    
    for ngram in n_grams:
        prefix = ngram[:-1]
        suffix = ngram[-1]
        ngrams_freq[prefix][suffix] += 1
        
    return ngrams_freq

# Создание N-граммной модели
ngrams_freq = generate_ngrams(tokens, n)
print(ngrams_freq)

defaultdict(<class 'collections.Counter'>, {('кошка',): Counter({'сидит': 1}), ('сидит',): Counter({'на': 1}), ('на',): Counter({'диване': 1, 'ковре': 1}), ('диване',): Counter({'собака': 1}), ('собака',): Counter({'лежит': 1}), ('лежит',): Counter({'на': 1})})


### Генерация текста

In [8]:
# Функция для генерации текста
def generate_text(ngrams_freq, n, length):
    # Выбор случайного начального префикса
    start = random.choice(list(ngrams_freq.keys()))
    result = list(start)
    
    for _ in range(length - len(start)):
        prefix = tuple(result[-(n-1):])
        suffixes = ngrams_freq[prefix]
        if suffixes:
            next_word = random.choices(list(suffixes.keys()), weights=suffixes.values())[0]
            result.append(next_word)
        else:
            break
        
    return ' '.join(result)

# Генерация текста длиной 50 слов
generated_text = generate_text(ngrams_freq, n, 4)
print("Сгенерированный текст:", generated_text)

Сгенерированный текст: кошка сидит на ковре
